<font size = 6>**Detect eye blinks using Pupil Labs' blink detection pipeline**

Detecting eye blinks accurately is a crucial step for many eye tracking applications. On the one hand, the (partial) occlusion of the pupil during eye blinks prevents (hinders) external visual information from entering the retina and, thus, being processed further up the visual hierarchy. In a laboratory setting, for instance, this can be important in order to exclude trials during which a human observer has had no access to the information they were presented with. The (partial) occlusion of the pupil during eye blinks additionally impairs gaze estimation, hence limiting the eye tracker's accuracy. On the other hand, blink statistics, such as blink rate or blink duration, can serve as relevant physiological, clinical, and cognitive parameters. The motivation behind accuratenly detecting eye blinks is therefore two-folds: identifying time periods contaminated by blink-related artifacts as well isolating relevant parameters. <br><br>

This Alpha Lab aims to demonstrate how the Pupil Labs blink detection algorithm works and how to use it. In the first part of the tutorial, we will have a look at offline blink detection. To this end, we will provide an example recording (made with a Neon device) that users can explore. In order to be also <b>accessible by users with little coding experience</b> we provide a high-level interface to the blink detection algorithm. In the second part of this Alpha Lab, we will demonstrate how to utilize the blink detection pipeline in order to <b>detect eye blinks in (quasi) real-time</b>. This section will also provide more in-depth insights into the algorithm and its individual components.<br><br>

The full code accompanying this notebook can be found here: github.com/pupil-labs/

If you want to read a more technical summary of the Pupil Labs blink detection algorithm, you can find a short white paper in the documentation: PROVIDE LINK TO DOCUMENTATION HERE

**<font size = 5>Part 1:** Offline blink detection

In the first part of this Alpha Lab, we will perform offline blink detection. This part is meant to be easily accessible, even with little coding experience. If you have no recording at hand, don't worry - we provide you with an example recording that you can explore. However, free to test the pipeline with your own recording.<br><br>
First, let's import the relevant modules:

In [1]:
from blink_detector.blink_detector import blink_detection_pipeline
from blink_detector.helper import process_recording
import numpy as np

Next, we specify the path to the example recording that we want to analyze. If you want to start exploring the example data set, no changes are needed here. In case you have your own recording that you would like to analyze, simply replace the recording path with a path to your own recording. 

In [3]:
recording_path = "blink_detector/data/padel_tennis_neon_03-2ded8f56"

The Pupil Labs blink detection pipeline computes optical flow from sequences of eye video frames (you can find all the details in the Documentation linked to above). To this end, we first need to extract the frames from the eye camera recording and apply some minor preprocessing to the data. Note that we also have to supply a parameter that specifies if the recording is from a Neon device or not. As the example recording is from a **Neon device**, we set the is_neon parameter to *True*. If you want to try out the blink detection pipeline with an **Invisible recording**, simply set this parameter to *False*. 

In [4]:
left_eye_images, right_eye_images, ts = process_recording(
    recording_path, is_neon=True
)

 In order to detect eye blinks in the recording, we can now call the blink_detection_pipeline function and pass the extracted left and right eye video frames as well as the corresponding timestamps along. Depending on your computer and the length of your recording (if you run the analysis on your own), this can take a while. The function returns a generator object which can be easily converted into a list of blink events:

In [5]:
blink_events = list(blink_detection_pipeline(left_eye_images, right_eye_images, ts))

**And that's it!**<br><br>
Now that we have extracted the dectected blink events, we can obtain some blink statistics that may be useful further down the line. In the following cell, we print a few of those statistics: the total number of detected blinks, the estimated blink rate (in Hz) as well as the average blink duration (in seconds). We can even divide this further up and look into the average durations of the eyelid closing and the re-opening events:

In [6]:
print(
    f"Number of blinks: {len(blink_events)}"

    f"\nBlink rate [in Hz]: {len(blink_events) / ((ts[-1] - ts[0])/1e9):.2f}"

    f"\nAvg. blink duration [in sec]: {np.mean([blink.blink_duration_s for blink in blink_events]):.3f}"

    f"\nAvg. eyelid closing duration [in sec]: {np.mean([blink.eyelid_closing_duration_s for blink in blink_events]):.3f}"

    f"\nAvg. eyelid opening duration [in sec]: {np.mean([blink.eyelid_opening_duration_s for blink in blink_events]):.3f}"
)

Number of blinks: 21
Blink rate [in Hz]: 0.58
Avg. blink duration [in sec]: 0.300
Avg. eyelid closing duration [in sec]: 0.131
Avg. eyelid opening duration [in sec]: 0.163


As you see, it is fairly straightforward to apply the blink detection pipeline and extract a number of interesting parameters. In case you took a closer look at the blink duration and the eyelid closing and re-opening times and were wondering why the total blink duration does not (always) equal the sum of the closing and re-opening events: this happens as for some frames between eyelid closing and subsequent re-opening, the optical flow is effectively zero and those are, thus, neither part of the closing or re-opening sequence. However, blink duration is computed from starting time of the closing of the eyelid to the the last frame that is classified as being part of the re-opening sequence. Hence, blink duration will always longer than the sum of the invidiual events.<br><br>
To end the first part of this tutorial, we will visualize the detected blinks in order to get an idea of their temporal distribution:

In [ ]:
from blink_detector.helper import visualize_blink_events

# Limit the visualization to the first 200 s (if the recording is longer than that)
max_duration = 200

visualize_blink_events(blink_events, ts, max_duration)

We can now see how the detected blinks (shown as green rectangles) are distributed over time and how they vary in duration.

<font size = 5><b>(2) Blink detection in real time using PL's realtime API</b></font>

As you have seen in Part 1, applying the blink detection pipeline to downloaded recordings is fairly straightforward. In the next part of this tutorial, we will show you how you can detect eye blinks in (quasi) real-time utilizing Pupil Lab's Real Time API. This could be useful in order to track blink rate over time (for example, serving as a biomarker of fatigue) or even use certain blink sequences as a means to control an external device. In the following sections, we will show you how to remotely start and stop recordings with your Neon or Invisible device by blinking three times in quick succession.<br><br>

Note that we the temporal resolution of the blink detection is limited by the optical flow parameters (i.e., the number of layers and the layer interval) and postprocessing parameters (e.g. the minimum blink duration). Moreover, depending on your computational resources, the demands of the image processing can further impede performance. Thus, there is an inevitable delay in the blink detection, rendering it only quasi real-time. <br><br>
**Note:** In order to run this second part of the notebook, it is necessary to download and install *Pupil Lab's Realtime Python API*. You can find the code on GitHub (https://github.com/pupil-labs/realtime-python-api) and detailed instructions on how to install and use the realtime API in the documentation (https://pupil-labs-realtime-api.readthedocs.io).

After you have successfully installed the Realtime API, let's first import all the required modules for the realtime blink detection:

In [1]:
import nest_asyncio
from pupil_labs.realtime_api.simple import discover_one_device, discover_devices
from blink_detector.helper import stream_images_and_timestamps
from blink_detector.blink_detector import (
    blink_detection_pipeline,
    successive_blink_detector,
)

Next, we need to setup the real time API. We use Python's *asyncio* to implement asynchronous communication. In some environments, for example when working with Jupyter notebooks, asyncio won't work out of the box. We will make use of *nest_asyncio*, which allows *asyncio* to work also within Jupyter notebooks. 

In [2]:
nest_asyncio.apply()

In [3]:
devices = discover_devices(5)
device = next(d for d in devices if "tom" in d.phone_name.lower())

Printing the device name and its IP helps to make sure that the discovered device is indeed <br>
the device you would like to connect with.

In [4]:
print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")

Phone IP address: 192.168.20.131
Phone name: Neon Companion Tom


Start and end Neon recording with 4 blinks in quick succession. Send event message to Neon with 3 blinks in quick succession.

In [ ]:
device_is_recording = False

left_images, right_images, timestamps = stream_images_and_timestamps(
    device, is_neon=True
)

detect_three_successive_blinks = successive_blink_detector(
    n_successive_blinks=3, 
)

for blink_events in blink_detection_pipeline(left_images, right_images, timestamps):

    toggle_recording = detect_three_successive_blinks(blink_events)

    if toggle_recording:
        if device_is_recording:
            print("Stop recording")
            device.recording_cancel()
            device_is_recording = False
        else:
            print("Start recording")
            device.recording_start()
            device_is_recording = True
        toggle_recording = False

**Real-time blink rate since start of recording and over the last 30s**

In this last part of the Alpha Lab, we will show you how t utilize the Realtime API in combination with the blink detection pipeline in order to perform realtime blink-rate estimation.

In [ ]:
import time
import numpy as np

# we need to re-initialize the streams as the previous ones have been exhausted
left_images, right_images, timestamps = stream_images_and_timestamps(
    device, is_neon=True
)

blink_counter = 0
starting_time = time.time()

# program breaks if you manage to blink more than 100 times within 30s
blink_timings = np.zeros(100)

for blink_events in blink_detection_pipeline(left_images, right_images, timestamps):

    blink_counter += 1

    blink_timings = np.roll(blink_timings, 1)
    blink_timings[0] = blink_events.start_time / 1e9 

    # compute how many blinks happened within the last 30s
    blink_counter = np.sum(
        (blink_timings > (time.time() - 30)) & (blink_timings < time.time())
    )

    total_blink_rate = blink_counter / (time.time() - starting_time)

    if time.time() - starting_time > 30:    
        last_30s_blink_rate = blink_counter / 30.0
    else:
        last_30s_blink_rate = total_blink_rate

    print('Blink rate (total) [in Hz]: %.3f ||| Blink rate (last 30s) [in Hz]: %.3f' 
    % (total_blink_rate, last_30s_blink_rate), end='\r')


In [ ]:
blink_events.start_time / 1e9 - starting_time

3.594893217086792

no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing


**(2.3) Set up real-time data stream**

In [3]:
import asyncio

async def loop():
    while True:
        print("gibberish")
        await asyncio.sleep(0.5)

async def main():
    future = asyncio.ensure_future(loop())
    for i in range(100):
        print("pass")
        await asyncio.sleep(1)
    future.cancel()
asyncio.get_event_loop().run_until_complete(main())    

NameError: name 'nest_asyncio' is not defined

**(2.4) Run blink detection pipeline**

Detect and print blink events in (quasi) real-time

In [6]:

# create three independent generator objects from the video stream
stream_left, stream_right, stream_ts = tee(video_stream(device), 3)

left_images = (left for left, _, _ in stream_left)
right_images = (right for _, right, _ in stream_right)

# timestamps need to be converted to ns
timestamps = (1e9 * timestamp for _, _, timestamp in stream_ts)

blink_event = None
import threading

def background():
    global blink_event
    while True:
        blink_event = next(blink_detection_pipeline(left_images, right_images, timestamps))
        return blink_event

def foreground():
    while True:
        print(blink_event)
        time.sleep(1)

bg = threading.Thread(target=background)
fg = threading.Thread(target=foreground)
bg.start()
fg.start()


None


Exception in thread Thread-6 (background):
Traceback (most recent call last):
  File "/Users/tpfeffer/opt/anaconda3/envs/py310/lib/python3.10/site-packages/pupil_labs/realtime_api/simple/device.py", line 203, in _receive_item
    return self._most_recent_item[sensor].popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tpfeffer/opt/anaconda3/envs/py310/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/tpfeffer/opt/anaconda3/envs/py310/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/6s/7t24f5ts6d51jy_w7578ts2h0000gn/T/ipykernel_21115/367240790.py", line 16, in background
  File "/Users/tpfeffer/projects/real-time-blink-detection/blink_detector/blink_detector.py", line 315, in extract_blink_events
    for event1, event2 in pairwise(onsets_offset_events):
  File "/Users/tp

None
None
None
None
None
None
None
None
None
BlinkEvent(start_time=1.6847659196974748e+18, end_time=1.684765919877597e+18, label='blink', blink_duration_s=0.180122112, eyelid_closing_duration_s=0.130121984, eyelid_opening_duration_s=0.045000192)
BlinkEvent(start_time=1.6847659196974748e+18, end_time=1.684765919877597e+18, label='blink', blink_duration_s=0.180122112, eyelid_closing_duration_s=0.130121984, eyelid_opening_duration_s=0.045000192)
BlinkEvent(start_time=1.6847659196974748e+18, end_time=1.684765919877597e+18, label='blink', blink_duration_s=0.180122112, eyelid_closing_duration_s=0.130121984, eyelid_opening_duration_s=0.045000192)
BlinkEvent(start_time=1.6847659196974748e+18, end_time=1.684765919877597e+18, label='blink', blink_duration_s=0.180122112, eyelid_closing_duration_s=0.130121984, eyelid_opening_duration_s=0.045000192)
BlinkEvent(start_time=1.6847659196974748e+18, end_time=1.684765919877597e+18, label='blink', blink_duration_s=0.180122112, eyelid_closing_duration_s=0.

In [2]:
from PyQt5 import QtWidgets, QtCore, QtGui
import sys
import numpy as np

class MainWindow(QtWidgets.QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.data = np.random.randint(0, 255, (100, 240, 320), dtype=np.uint8)

        self.label = QtWidgets.QLabel(self)
        self.setCentralWidget(self.label)

        self.frame_num = 0

        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(50)  # update every 100 ms

    def update_frame(self):
        frame = self.data[self.frame_num, :, :]

        # Convert the numpy array to a QImage
        qimg = QtGui.QImage(frame.data, frame.shape[1], frame.shape[0], 
                            frame.strides[0], QtGui.QImage.Format_Grayscale8)

        # Convert the QImage to a QPixmap and show it
        self.label.setPixmap(QtGui.QPixmap.fromImage(qimg))
        self.frame_num = (self.frame_num + 1) % self.data.shape[0]


app = QtWidgets.QApplication(sys.argv)
w = MainWindow()
w.show()
sys.exit(app.exec_())


: 

: 

In [ ]:
device.receive_eyes_video_frame()

In [36]:
import numpy as np
np.random.randint(0, 2)

0

0
1
2
3
4
